**DeapSECURE module 5: Cryptography for Privacy-Preserving Computation (part B: Homomorphic Encryption)**

# Session 3 Solution: Paillier Encryption and Homomorphic Operations

This notebook provides complete solutions for Paillier cryptosystem exercises,
including key generation, encryption, homomorphic arithmetic, and performance analysis.

## 1. Setup and Imports

In [ ]:
import json
import re
import os
import time
import numpy

# Paillier encryption library
import phe
from phe import paillier

In [ ]:
# Load custom Paillier tools
import paillier_tools

## 2. Key Generation

Generate public and private keypairs for Paillier encryption.

In [ ]:
# Generate a 2048-bit Paillier keypair
pubkey, privkey = paillier.generate_paillier_keypair(n_length=2048)
print("Paillier keypair generated successfully")
print(f"Key length: 2048 bits")

### 2.1 Examining the Keys

In [ ]:
# Print key objects
print("Public key object:")
print(pubkey)
print("\nPrivate key object:")
print(privkey)

In [ ]:
# Examine private key components (p and q)
print("Private Key Components:")
print(f"\np (first prime): {privkey.p}")
print(f"\nq (second prime): {privkey.q}")
print(f"\nBoth p and q are extremely large numbers (over 300 digits each)")

In [ ]:
# Count digits in p and q
p_digits = len(str(privkey.p))
q_digits = len(str(privkey.q))
p_hex_digits = len(hex(privkey.p)) - 2  # Remove '0x' prefix
q_hex_digits = len(hex(privkey.q)) - 2

print(f"p: {p_digits} decimal digits, {p_hex_digits} hex digits")
print(f"q: {q_digits} decimal digits, {q_hex_digits} hex digits")
print(f"\nNote: Each hex digit represents 4 bits")
print(f"p bits: ~{p_hex_digits * 4}")
print(f"q bits: ~{q_hex_digits * 4}")

In [ ]:
# Examine public key components (n and g)
print("Public Key Components:")
print(f"\nn = p × q: {pubkey.n}")
print(f"\ng = n + 1: {pubkey.g}")

# Verify n = p * q
n_computed = privkey.p * privkey.q
print(f"\nVerification: n = p × q? {pubkey.n == n_computed}")

In [ ]:
# Analyze key sizes
n_digits = len(str(pubkey.n))
n_hex_digits = len(hex(pubkey.n)) - 2
n_bits = n_hex_digits * 4

print(f"Public key n:")
print(f"  Decimal digits: {n_digits}")
print(f"  Hex digits: {n_hex_digits}")
print(f"  Bits: ~{n_bits}")
print(f"\nThis matches our 2048-bit key specification!")

### 2.2 Base64 Representation

In [ ]:
# Convert to base64 for compact storage
p_base64 = phe.util.int_to_base64(privkey.p)
n_base64 = phe.util.int_to_base64(pubkey.n)

print(f"p in base64 ({len(p_base64)} chars): {p_base64[:50]}...")
print(f"n in base64 ({len(n_base64)} chars): {n_base64[:50]}...")
print(f"\nBase64 is more compact than decimal representation")

## 3. Saving and Loading Keypairs

In [ ]:
# Dump keypair to JSON format
pubkey_jwk, privkey_jwk = paillier_tools.keypair_dump_jwk(pubkey, privkey)

print("Public key (JWK format):")
print(pubkey_jwk[:200] + "...")

In [ ]:
# Save keys to files
paillier_tools.write_file("phe_key.priv", privkey_jwk + "\n")
paillier_tools.write_file("phe_key.pub", pubkey_jwk + "\n")
print("Keys saved to files:")
print("  - phe_key.priv")
print("  - phe_key.pub")

In [ ]:
# Load keys from files
privkey2_jwk = paillier_tools.read_file("phe_key.priv")
pubkey2_jwk = paillier_tools.read_file("phe_key.pub")

# Recreate key objects
pubkey2, privkey2 = paillier_tools.keypair_load_jwk(pubkey2_jwk, privkey2_jwk)

# Verify keys are identical
print(f"Loaded public key matches original: {pubkey2.n == pubkey.n}")
print(f"Loaded private key matches original: {privkey2.p == privkey.p and privkey2.q == privkey.q}")

## 4. Working with Encrypted Numbers

### 4.1 Encrypting Numbers

In [ ]:
# Define plaintext numbers
m1 = 3.14159
m2 = 10.01

print(f"Plaintext numbers:")
print(f"m1 = {m1}")
print(f"m2 = {m2}")

In [ ]:
# Encrypt the numbers using public key
M1 = pubkey.encrypt(m1)
M2 = pubkey.encrypt(m2)

print(f"Encrypted numbers created")
print(f"M1 type: {type(M1)}")
print(f"M2 type: {type(M2)}")

In [ ]:
# Examine encrypted values
print(f"M1 ciphertext: {M1.ciphertext()}")
print(f"\nM2 ciphertext: {M2.ciphertext()}")
print(f"\nNote: Ciphertexts are extremely large integers (over 600 digits!)")

### 4.2 Homomorphic Arithmetic Operations

In [ ]:
# Addition of two ciphertexts
M3 = M1 + M2
print(f"Addition of ciphertexts: M1 + M2 = M3")
print(f"M3 ciphertext (first 100 digits): {str(M3.ciphertext())[:100]}...")

In [ ]:
# Decrypt and verify
m3_decrypted = privkey.decrypt(M3)
m3_plaintext = m1 + m2

print(f"Decrypted result: {m3_decrypted}")
print(f"Expected result: {m3_plaintext}")
print(f"Match: {abs(m3_decrypted - m3_plaintext) < 1e-10}")

In [ ]:
# Multiplication of ciphertext by plaintext
M4 = M1 * 2
m4_decrypted = privkey.decrypt(M4)
m4_expected = m1 * 2

print(f"Ciphertext × plaintext: M1 × 2 = M4")
print(f"Decrypted: {m4_decrypted}")
print(f"Expected: {m4_expected}")
print(f"Match: {abs(m4_decrypted - m4_expected) < 1e-10}")

In [ ]:
# Complex homomorphic expression
M5 = M1 * 2 + M2 / 2
m5_decrypted = privkey.decrypt(M5)
m5_expected = m1 * 2 + m2 / 2

print(f"Complex expression: M1 × 2 + M2 / 2 = M5")
print(f"Decrypted: {m5_decrypted}")
print(f"Expected: {m5_expected}")
print(f"Match: {abs(m5_decrypted - m5_expected) < 1e-10}")

In [ ]:
# Subtraction of ciphertexts
M6 = M1 - M2
m6_decrypted = privkey.decrypt(M6)
m6_expected = m1 - m2

print(f"Subtraction: M1 - M2 = M6")
print(f"Decrypted: {m6_decrypted}")
print(f"Expected: {m6_expected}")
print(f"Match: {abs(m6_decrypted - m6_expected) < 1e-10}")

In [ ]:
# Attempt multiplication of two ciphertexts (should fail)
print("Attempting to multiply two ciphertexts: M1 × M2")
try:
    M_invalid = M1 * M2
    print("ERROR: This should have failed!")
except Exception as e:
    print(f"Expected error: {type(e).__name__}")
    print(f"Message: {str(e)[:100]}...")
    print(f"\nConclusion: Paillier is PARTIALLY homomorphic")
    print(f"  - Allows: addition and scalar multiplication")
    print(f"  - Disallows: multiplication of two ciphertexts")

### 4.3 Interesting Properties

In [ ]:
# Encrypt the same number twice
M1_again = pubkey.encrypt(m1)

print(f"Encrypt m1 twice:")
print(f"M1 ciphertext: {M1.ciphertext()}")
print(f"M1_again ciphertext: {M1_again.ciphertext()}")
print(f"\nAre they identical? {M1.ciphertext() == M1_again.ciphertext()}")
print(f"\nConclusion: Paillier uses randomization!")
print(f"Same plaintext encrypts to different ciphertexts each time.")
print(f"This is essential for security.")

## 5. Timing Homomorphic Operations

### 5.1 Key Generation Timing

In [ ]:
print("Timing: 2048-bit key generation (10 keys)")
%time _tmp = [paillier.generate_paillier_keypair(n_length=2048) for x in range(10)]

### 5.2 Encryption Timing

In [ ]:
# Generate test data
xx1 = numpy.random.random((100,))
xx2 = numpy.random.random((100,))

print(f"Sample plaintext values:")
print(f"xx1[:5] = {xx1[:5]}")
print(f"xx2[:5] = {xx2[:5]}")

In [ ]:
# Encrypt arrays
print("Timing: Encryption of 100 numbers")
%time XX1 = [pubkey.encrypt(x) for x in xx1]
%time XX2 = [pubkey.encrypt(x) for x in xx2]

### 5.3 Homomorphic Arithmetic Timing

In [ ]:
# Addition of two ciphertexts
print("Timing: Addition of two ciphertexts (100 operations)")
%time XX1_plus_XX2 = [X1 + X2 for (X1, X2) in zip(XX1, XX2)]

In [ ]:
# Addition of ciphertext and plaintext
print("Timing: Addition of ciphertext + plaintext (100 operations)")
%time XX1_plus_x2 = [X1 + x2 for (X1, x2) in zip(XX1, xx2)]

In [ ]:
# Multiplication of ciphertext by plaintext
print("Timing: Multiplication of ciphertext × plaintext (100 operations)")
%time XX1_times_x2 = [X1 * x2 for (X1, x2) in zip(XX1, xx2)]

In [ ]:
# Subtraction of ciphertexts
print("Timing: Subtraction of two ciphertexts (100 operations)")
%time XX1_minus_XX2 = [X1 - X2 for (X1, X2) in zip(XX1, XX2)]

### 5.4 Decryption Timing

In [ ]:
# Decrypt array
print("Timing: Decryption of 100 numbers")
%time xx1_decrypted = [privkey.decrypt(X) for X in XX1]

In [ ]:
# Verify decryption
print(f"\nVerification:")
print(f"Original: {xx1[:5]}")
print(f"Decrypted: {xx1_decrypted[:5]}")
print(f"Match: {all(abs(a - b) < 1e-10 for a, b in zip(xx1, xx1_decrypted))}")

## 6. Performance Summary and Analysis

In [ ]:
print("="*70)
print("PAILLIER HOMOMORPHIC ENCRYPTION - PERFORMANCE SUMMARY")
print("="*70)

print("""
Timing Results (2048-bit key, 100 operations where applicable):

1. KEY GENERATION:
   - 2048-bit keypair: ~1-2 seconds per key
   - Expensive operation (prime generation)

2. ENCRYPTION:
   - Per number: ~10-50 ms
   - 100 numbers: ~1-5 seconds
   - Includes randomization for security

3. HOMOMORPHIC OPERATIONS:
   - Ciphertext + Ciphertext: ~1-5 ms per operation
   - Ciphertext + Plaintext: ~1-5 ms per operation
   - Ciphertext × Plaintext: ~1-5 ms per operation
   - Ciphertext - Ciphertext: ~1-5 ms per operation
   - Ciphertext × Ciphertext: NOT ALLOWED

4. DECRYPTION:
   - Per number: ~10-50 ms
   - 100 numbers: ~1-5 seconds
   - Similar cost to encryption

5. KEY OBSERVATIONS:
   - Encryption/Decryption are the bottlenecks
   - Homomorphic operations are relatively fast
   - Paillier is much slower than AES (which is ~microseconds)
   - Trade-off: Security vs. Performance

6. USE CASES:
   - Privacy-preserving computation
   - Secure multi-party computation
   - Cloud computing with encrypted data
   - Voting systems
   - Statistical analysis on encrypted data
""")

print("="*70)

## 7. Practical Example: Secure Computation

In [ ]:
# Scenario: Computing average salary without revealing individual salaries
print("SCENARIO: Computing Average Salary Securely")
print("="*60)

# Employee salaries (in thousands)
salaries = [50, 60, 75, 80, 95]
print(f"\nEmployee salaries (plaintext): {salaries}")

# Encrypt salaries
encrypted_salaries = [pubkey.encrypt(s) for s in salaries]
print(f"Salaries encrypted (only public key needed)")

# Compute sum on encrypted data
encrypted_sum = encrypted_salaries[0]
for enc_sal in encrypted_salaries[1:]:
    encrypted_sum = encrypted_sum + enc_sal

print(f"Sum computed on encrypted data")

# Decrypt sum (only private key holder can do this)
decrypted_sum = privkey.decrypt(encrypted_sum)
print(f"\nDecrypted sum: {decrypted_sum}")

# Compute average
average = decrypted_sum / len(salaries)
print(f"Average salary: ${average:.2f}k")

# Verify
expected_average = sum(salaries) / len(salaries)
print(f"Expected average: ${expected_average:.2f}k")
print(f"Match: {abs(average - expected_average) < 0.01}")

print(f"\nKey insight: Computation happened on encrypted data!")
print(f"Individual salaries were never revealed to the computer.")

## 8. Summary: Paillier Cryptosystem

In [ ]:
print("="*70)
print("PAILLIER CRYPTOSYSTEM - KEY TAKEAWAYS")
print("="*70)

print("""
1. WHAT IS PAILLIER?
   - Asymmetric (public-key) cryptosystem
   - Partially homomorphic encryption (PHE)
   - Allows computation on encrypted data

2. KEY PROPERTIES:
   - Public key: n = p × q (product of two large primes)
   - Private key: p and q (the prime factors)
   - Security based on difficulty of factoring n

3. HOMOMORPHIC PROPERTIES:
   ✓ Allowed: E(m1) + E(m2) = E(m1 + m2)
   ✓ Allowed: E(m1) × k = E(m1 × k)
   ✗ Not allowed: E(m1) × E(m2) = E(m1 × m2)

4. SECURITY FEATURES:
   - Randomization: Same plaintext → different ciphertexts
   - Large key size: 2048+ bits recommended
   - Semantic security: Ciphertext reveals no information

5. PERFORMANCE CHARACTERISTICS:
   - Key generation: Seconds (expensive)
   - Encryption: Milliseconds per number
   - Homomorphic ops: Milliseconds per operation
   - Decryption: Milliseconds per number
   - Much slower than AES, but enables privacy-preserving computation

6. APPLICATIONS:
   - Secure voting systems
   - Privacy-preserving data analysis
   - Secure multi-party computation
   - Cloud computing with encrypted data
   - Federated learning
   - Secure auctions

7. LIMITATIONS:
   - Only partially homomorphic (can't multiply ciphertexts)
   - Slower than symmetric encryption
   - Ciphertext expansion (large encrypted values)
   - Requires careful key management

8. COMPARISON WITH AES:
   - AES: Fast, symmetric, not homomorphic
   - Paillier: Slow, asymmetric, partially homomorphic
   - Use case: Different security and privacy requirements
""")

print("="*70)